# ライブラリ

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# import japanize_matplotlib
import seaborn as sns


from function import create_data

# データ読みこみ

In [2]:
category_names_csv = '../original_data/category_names.csv'
item_categories_csv = '../original_data/item_categories.csv'
sales_history1_csv = '../original_data/sales_history1.csv'
sales_history2_csv = '../original_data/sales_history2.csv'

# DataFrame作成

## Category_name作成

In [3]:
df_category_name = pd.read_csv(category_names_csv, encoding='shift_jis')

- 『商品カテゴリ名』=> 『商品カテゴリ』『商品名』に変換

In [4]:
df_category_name = create_data.split_column_by_delimiter(df_category_name, '商品カテゴリ名', '-', ['商品カテゴリ', '商品名'])

- 商品カテゴリでグループ分け

In [5]:
# 商品カテゴリ列が同じものの中で、商品名列のユニーク値を確認する
result = create_data.check_unique_values(df_category_name, '商品カテゴリ', '商品名')
print(result)

商品カテゴリ
PCゲーム                                                  [通常版]
PCソフト                                                [PCソフト]
ギフト        [小型ゲーム, カード、パズル等, フィギュア, ぬいぐるみ, ボードゲーム, ガジェット類...
ゲーム                         [ゲーム機C, ゲーム機B, ゲーム機A, 付属品(購入特典)]
ゲーム周辺機器                                       [ゲーム機B, ゲーム機A]
映画                              [DVD, ブルーレイ, ブルーレイ3D, セット商品]
本                        [オーディオブック_タイプA, 教育用, オーディオブック_タイプB]
音楽                                           [その他, CD, レコード]
Name: 商品名, dtype: object


## Item_categories作成

In [6]:
item_categories = pd.read_csv(item_categories_csv, encoding='shift_jis')

## Sales_history作成

In [7]:
df_sales_history1 = pd.read_csv(sales_history1_csv, encoding='shift_jis')
df_sales_history2 = pd.read_csv(sales_history2_csv, encoding='shift_jis')

- 合成

In [8]:
# カラム名は等しいので、縦に結合+index初期化
df_sales_history = pd.concat([df_sales_history1, df_sales_history2], ignore_index=True)
df_sales_history.sort_values(by='日付', inplace=True)
df_sales_history.reset_index(drop=True, inplace=True)

df_sales_history['売上'] = df_sales_history['商品価格'] * df_sales_history['売上個数']

- 出力

In [9]:
df_sales_history.to_csv('../processing_data/sales_history.csv', index=False, encoding='shift_jis')

# データ結合

category_names + item_categories + sales_history<br>
主キーは、商品カテゴリ

- category_names + item_categories

In [10]:
# '商品カテゴリID'でマージする
df_temp = pd.merge(item_categories, df_category_name, on='商品カテゴリID', how='inner')

- + sales_history

In [11]:
merged_df = pd.merge(df_sales_history, df_temp, on='商品ID', how='inner')

- ソート

In [12]:
merged_df.sort_values(by='日付', inplace=True)
merged_df.reset_index(drop=True, inplace=True)

- 曜日列作成

In [14]:
# 日付列をdatetime型に変換
merged_df['日付'] = pd.to_datetime(merged_df['日付'])

# 日付から曜日列を作成
merged_df['曜日'] = merged_df['日付'].dt.day_name()

- 年月日列作成

In [15]:
merged_df['年'] = merged_df['日付'].dt.year
merged_df['月'] = merged_df['日付'].dt.month
merged_df['日'] = merged_df['日付'].dt.day

- 四半期列作成

In [16]:
merged_df['四半期'] = merged_df['日付'].dt.quarter

- 週の列作成

In [17]:
merged_df['週'] = merged_df['日付'].dt.isocalendar().week

- 平日/休日の列作成

In [18]:
merged_df['平日/休日'] = merged_df['日付'].dt.dayofweek // 5  # 0が平日、1が休日

- 出力

In [19]:
merged_df.to_csv('../processing_data/merged.csv', encoding='shift_jis', index=False)

In [20]:
merged_df

,日付,店舗ID,商品ID,商品価格,売上個数,売上,商品カテゴリID,商品カテゴリ名,商品カテゴリ,商品名,曜日,年,月,日,四半期,週,平日/休日
0,2018-01-01,6,1000001,420,1,420,100,映画 - DVD,映画,DVD,Monday,2018,1,1,1,1,0
1,2018-01-01,1,3100018,1950,1,1950,310,ゲーム - 付属品(購入特典),ゲーム,付属品(購入特典),Monday,2018,1,1,1,1,0
2,2018-01-01,7,2200016,510,1,510,220,ゲーム - ゲーム機A,ゲーム,ゲーム機A,Monday,2018,1,1,1,1,0
3,2018-01-01,10,2400001,2850,1,2850,240,ギフト - ボードゲーム,ギフト,ボードゲーム,Monday,2018,1,1,1,1,0
4,2018-01-01,15,1500067,3230,1,3230,150,ゲーム - ゲーム機B,ゲーム,ゲーム機B,Monday,2018,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-31,7,1100443,1020,2,2040,110,PCゲーム - 通常版,PCゲーム,通常版,Thursday,2019,10,31,4,44,0
1119566,2019-10-31,7,1100443,1020,1,1020,110,PCゲーム - 通常版,PCゲーム,通常版,Thursday,2019,10,31,4,44,0
1119567,2019-10-31,8,1600060,1020,1,1020,160,ギフト - 小型ゲーム,ギフト,小型ゲーム,Thursday,2019,10,31,4,44,0
1119568,2019-10-31,3,1002540,680,1,680,100,映画 - DVD,映画,DVD,Thursday,2019,10,31,4,44,0
